### Example 2: extensions
More complicated inputs. .
1. Here, we want to pass in an array to the inference.
2. Notice that the warmup example fixes the lengths of the array that the onnx model expects.
3. check out the netron visualization of the model.

In this example, we take an array `x`, and then take the `max` of the entries in `x`.
denote the max by `k`, then we return `m*k + c`

In [194]:
onnx_model = "example2.onnx"

In [195]:
import torch
import torch.nn as nn
import numpy as np

In [196]:
class SampleNet(nn.Module):
    def __init__(self):
        super(SampleNet, self).__init__()
        self.m = 2
        self.c = 1
    
    def forward(self, x):
        return self.m * torch.max(x) + self.c

### note
if we replace the `torch.max` above by `np.max` then we will get errors

In [197]:
model = SampleNet()
model.eval()

SampleNet()

In [198]:
# the warmup stage. 
x = torch.LongTensor([1, 2, 3, 4])
out = model(x)

In [199]:
out

tensor(9)

In [200]:
torch.onnx.export(
model,
x,
onnx_model,
opset_version=11,
do_constant_folding=True,
input_names=['input'],
output_names=['output'])

In [201]:
# uncomment to install netron.
#!pip install netron
import netron
netron.start(onnx_model, port=8081)


Stopping http://localhost:8081
Serving 'example2.onnx' at http://localhost:8081


### use the onnx model

In [202]:
import onnxruntime as ort
import numpy as np
sess = ort.InferenceSession(onnx_model)

In [203]:
sess.get_inputs()[0].name

'input'

In [204]:
# check out the signature of sess.run - it has to have the output, then something like a feed_dict.

# either pass the feed dict directly.
# passing an array of any other size will result in an error.
outs = sess.run(['output'],
                       {
                         'input': np.array([11, 20, 22, 12], dtype=np.int64)  
                       })

print(outs)

[array(45, dtype=int64)]
